In [41]:
import ast
import re
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
import os
from joblib import dump, load

In [42]:
# ========== 0) Загрузка ==========
def load_df(path: str = ".data/train.csv") -> pd.DataFrame:
    """Ожидает CSV с колонками: sample;annotation"""
    df = pd.read_csv(path, sep=";")
    # приводим к строкам
    df["sample"] = df["sample"].astype(str)
    df["annotation"] = df["annotation"].astype(str)
    return df


# ========== 1) Парсер annotation ==========
def parse_annotation(cell: str) -> List[Tuple[int, int, str]]:
    """'[(0, 7, \"B-TYPE\"), ...]' -> [(0,7,'B-TYPE'), ...]; ничего не нормализуем."""
    if not isinstance(cell, str) or not cell.strip():
        return []
    try:
        items = ast.literal_eval(cell)
        return [(int(s), int(e), str(t)) for s, e, t in items]
    except Exception:
        return []


# ========== 2) Токенизация с индексами ==========
@dataclass
class Token:
    text: str
    start: int  # включительно
    end: int    # исключительно

_TOKEN_RE = re.compile(r"\S+")

def tokenize_with_offsets(text: str) -> List[Token]:
    """'абрикосы 500 г' -> [Token('абрикосы',0,8), Token('500',9,12), Token('г',13,14)]"""
    return [Token(m.group(0), m.start(), m.end()) for m in _TOKEN_RE.finditer(text or "")]


# ========== 3) Спаны -> BIO (по токенам) ==========
def spans_to_token_bio(tokens: List[Token],
                       spans: List[Tuple[int, int, str]]) -> List[str]:
    """
    Раздаём каждой позиции-токену BIO-метку из символьных спанов.
    Правило пересечения: >= 1 символа.
    """
    labels = ["O"] * len(tokens)
    for s, e, tag in spans:
        ent_type = tag.split("-", 1)[-1] if "-" in tag else tag
        first = True
        for i, t in enumerate(tokens):
            if not (t.end <= s or e <= t.start):
                labels[i] = f"{'B' if first else 'I'}-{ent_type}"
                first = False
    return labels


# ========== 4) Признаки токена (окно ±2) ==========
def _shape(w: str) -> str:
    return "".join("X" if c.isalpha() else "d" if c.isdigit() else "_" for c in w)

def token_features(tokens: List[Token], i: int) -> Dict[str, Any]:
    """Базовые фичи токена + контекст ±2; без POS/DEP и без словарей."""
    w = tokens[i].text
    wl = w.lower()
    feats: Dict[str, Any] = {
        "w.lower": wl,
        "shape": _shape(w),
        "is_digit": w.isdigit(),
        "has_digit": any(c.isdigit() for c in w),
        "has_pct": ("%" in w) or ("процент" in wl),
        "has_hyphen": "-" in w,
        "has_dot": "." in w,
        "has_comma": "," in w,
        "is_latin": bool(re.search(r"[A-Za-z]", w)),
        "is_cyrillic": bool(re.search(r"[А-Яа-яЁё]", w)),
        "is_upper": w.isupper(),
        "is_title": w.istitle(),
        "len": len(w),
        "len_bin": 0 if len(w) <= 2 else 1 if len(w) <= 5 else 2 if len(w) <= 10 else 3,
        "BOS": i == 0,
        "EOS": i == len(tokens) - 1,
    }
    # окно контекста: -1, -2, +1, +2
    def add_ctx(j: int, prefix: str):
        wj = tokens[j].text
        feats.update({
            f"{prefix}:w.lower": wj.lower(),
            f"{prefix}:shape": _shape(wj),
            f"{prefix}:is_upper": wj.isupper(),
            f"{prefix}:has_digit": any(c.isdigit() for c in wj),
            f"{prefix}:is_latin": bool(re.search(r"[A-Za-z]", wj)),
            f"{prefix}:is_cyr": bool(re.search(r"[А-Яа-яЁё]", wj)),
        })
    if i - 1 >= 0: add_ctx(i - 1, "-1")
    if i - 2 >= 0: add_ctx(i - 2, "-2")
    if i + 1 < len(tokens): add_ctx(i + 1, "+1")
    if i + 2 < len(tokens): add_ctx(i + 2, "+2")
    return feats

def sent2features(tokens: List[Token]) -> List[Dict[str, Any]]:
    return [token_features(tokens, i) for i in range(len(tokens))]


# ========== 5) CRF ==========
def init_crf(algorithm="lbfgs",
        c1=0.01,
        c2=0.05,
        max_iterations=150,
        all_possible_transitions=True,
        all_possible_states=False,
        verbose=False,) -> CRF:
    """CRF для BIO, 300 итераций."""
    return CRF(
        algorithm=algorithm,
        c1=c1,
        c2=c2,
        max_iterations=max_iterations,
        all_possible_transitions=all_possible_transitions,
        all_possible_states=all_possible_states,
        verbose=verbose,
    )

def build_xy(df: pd.DataFrame):
    """Из df(sample, annotation) делает X (фичи по токенам) и y (BIO по токенам)."""
    X, y = [], []
    for row in df.itertuples(index=False):
        text = str(row.sample)
        spans = parse_annotation(row.annotation)
        toks = tokenize_with_offsets(text)
        X.append(sent2features(toks))
        y.append(spans_to_token_bio(toks, spans))
    return X, y

def fit_crf(crf: CRF, X, y) -> CRF:
    crf.fit(X, y)
    return crf


# ========== 6) BIO -> символьные спаны ==========
def bio_to_spans(tokens: List[Token], labels: List[str]) -> List[Tuple[int, int, str]]:
    """Склеиваем предсказанные BIO в интервалы (start,end,'B-ENTITY') в координатах исходного текста."""
    spans = []
    cur_ent, cur_s, cur_e = None, None, None

    def push():
        if cur_ent is not None and cur_s is not None and cur_e is not None:
            spans.append((cur_s, cur_e, f"B-{cur_ent}"))

    for tok, lab in zip(tokens, labels):
        if not lab or lab == "O" or "-" not in lab:
            if cur_ent is not None:
                push()
                cur_ent, cur_s, cur_e = None, None, None
            continue
        pref, ent = lab.split("-", 1)
        if pref == "B":
            if cur_ent is not None:
                push()
            cur_ent, cur_s, cur_e = ent, tok.start, tok.end
        elif pref == "I":
            if cur_ent == ent:
                cur_e = tok.end
            else:
                if cur_ent is not None:
                    push()
                cur_ent, cur_s, cur_e = ent, tok.start, tok.end
    if cur_ent is not None:
        push()
    return spans


# ========== 7) Инференс одной строки ==========
def predict_one(crf: CRF, text: str) -> List[Tuple[int, int, str]]:
    toks = tokenize_with_offsets(text)
    if not toks:
        return []
    y_hat = crf.predict([sent2features(toks)])[0]
    return bio_to_spans(toks, y_hat)


# ========== 8) Метрика strict spans macro-F1 ==========
def spans_exact_f1(y_true: List[List[Tuple[int,int,str]]],
                   y_pred: List[List[Tuple[int,int,str]]]) -> float:
    """TP/FP/FN считаем по ПОЛНОМУ совпадению (start,end, type). Усреднение по TYPE/BRAND/VOLUME/PERCENT."""
    types = ["TYPE", "BRAND", "VOLUME", "PERCENT"]
    f1s = []
    for ent in types:
        tp = fp = fn = 0
        for t_sp, p_sp in zip(y_true, y_pred):
            T = {(s, e) for s, e, tag in t_sp if str(tag).endswith(ent)}
            P = {(s, e) for s, e, tag in p_sp if str(tag).endswith(ent)}
            I = T & P
            tp += len(I); fp += len(P - I); fn += len(T - I)
        prec = tp / (tp + fp) if (tp + fp) else 0.0
        rec  = tp / (tp + fn) if (tp + fn) else 0.0
        f1   = 2 * prec * rec / (prec + rec) if (prec + rec) else 0.0
        f1s.append(f1)
    return float(np.mean(f1s))


# ========== 9) Утилита для просмотра ==========
def spans_text_view(text: str, spans):
    parts = []
    for s, e, t in sorted(spans, key=lambda z: (z[0], z[1])):
        frag = text[s:e].replace("\n", "\\n")
        ent = t.split("-", 1)[-1] if "-" in t else t
        parts.append(f"{ent}='{frag}'[{s}:{e}]")
    return " | ".join(parts)

In [43]:
def save_model(crf: CRF, path: str, extra_meta: Dict[str, Any] | None = None) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    payload = {
        "model": crf,
        "meta": {
            "algo": "CRF-lbfgs",
            "c1": crf.c1, "c2": crf.c2,
            "max_iter": crf.max_iterations,
            "all_possible_transitions": crf.all_possible_transitions,
            **(extra_meta or {}),
        }
    }
    dump(payload, path)

def load_model(path: str) -> CRF:
    payload = load(path)
    return payload["model"]


In [44]:
TRAIN_PATH = ".data/train.csv"
MODEL_PATH = "models/crf_baseline_2.joblib" 

df = load_df(".data/train.csv")
df = df[df["sample"].str.len() > 0].reset_index(drop=True)

tr_df, va_df = train_test_split(df, test_size=0.15, random_state=42, shuffle=True)

X_tr, y_tr = build_xy(tr_df)
X_va, y_va = build_xy(va_df)

In [51]:
crf = init_crf(max_iterations=300)
fit_crf(crf, X_tr, y_tr)
save_model(crf, MODEL_PATH)

In [52]:
y_true_sp = [parse_annotation(a) for a in va_df["annotation"]]
y_pred_sp = [predict_one(crf, s) for s in va_df["sample"]]
f1 = spans_exact_f1(y_true_sp, y_pred_sp)
print(f"Validation macro-F1 (strict spans): {f1:.4f}")

Validation macro-F1 (strict spans): 0.7843


In [53]:
def spans_list(text: str, spans):
    """[(s,e,tag)] -> [(s, e, 'TAG')] с сохранёнными тегами как есть."""
    # просто убеждаемся, что это список кортежей правильного вида
    out = []
    for s, e, t in spans:
        out.append((int(s), int(e), str(t)))
    return out

def print_sample(text: str, gold_spans, pred_spans):
    print(f"sample: {text}")
    print(f"y: {spans_list(text, gold_spans)}")
    print(f"pred: {spans_list(text, pred_spans)}")


In [54]:
print("\n=== Samples ===")
for i in range(min(10, len(va_df))):
    text = va_df.iloc[i]["sample"]
    gold = y_true_sp[i]
    pred = y_pred_sp[i]
    print("-" * 70)
    print_sample(text, gold, pred)



=== Samples ===
----------------------------------------------------------------------
sample: лсвежитель
y: [(0, 10, 'B-TYPE')]
pred: [(0, 10, 'B-TYPE')]
----------------------------------------------------------------------
sample: варенец останкинск
y: [(0, 7, 'B-TYPE'), (8, 18, 'B-BRAND')]
pred: [(0, 7, 'B-TYPE'), (8, 18, 'B-BRAND')]
----------------------------------------------------------------------
sample: кабачковая икра
y: [(0, 10, 'B-TYPE'), (11, 15, 'I-TYPE')]
pred: [(0, 10, 'B-TYPE'), (11, 15, 'B-TYPE')]
----------------------------------------------------------------------
sample: фитики
y: [(0, 6, 'B-TYPE')]
pred: [(0, 6, 'B-TYPE')]
----------------------------------------------------------------------
sample: ресень
y: [(0, 6, 'B-TYPE')]
pred: [(0, 6, 'B-TYPE')]
----------------------------------------------------------------------
sample: столичный салато
y: [(0, 9, 'B-BRAND'), (10, 16, 'B-TYPE')]
pred: [(0, 9, 'B-BRAND'), (10, 16, 'B-TYPE')]
------------------------